<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from gen_n2v import getNode2Vec
import itertools
import os
import multiprocessing as mp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
n_walks = [10, 50, 100]
l_walk = [5, 20, 60]
dim = [8, 16, 64]
q = [0.1, 0.4, 0.8]
comb = [q, n_walks, l_walk, dim]
comb = list(itertools.product(*comb))

In [3]:
def calc(option, i):
    getter = getNode2Vec(path='./data/Caltech36/Caltech36',
                         q=option[0],
                         num_walks=option[1],
                         walk_length=option[2],
                         dim=option[3])
    getter.get_n2v(_id=i)

In [4]:
def get_dirs(path):
    dirs = []
    return [i for i in os.listdir(path)]



def get_data(name,sep=','):
    df = pd.DataFrame()
    for x in get_dirs():
        path = "./data/{}/{}{}".format(x, x, name)
        temp = pd.read_csv(path,sep=sep)
        df = df.append(temp)
    return df

In [5]:
path = './data/Caltech36/Caltech36comb/'
def logreg(data,i):
    score = []
    for x in range(10):
        X = data.iloc[:,:-1].values
        y = data['label'].values
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(X)
        X = scaler.transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        model = RandomForestClassifier().fit(X_train, y_train)
        yhat = model.predict(X_test)
        #print(confusion_matrix(y_test, yhat))

        #print(classification_report(y_test, yhat))
        #print(f'{i}::ROC_AUC_SCORE:',roc_auc_score(y_test, model.decision_function(X_test)))
        score.append(roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))
    return {"id":i,"score":np.mean(score)}

In [6]:
res = []
for i,file in enumerate(get_dirs(path)):
    data = pd.read_csv(path + file, sep=',')
    res.append(logreg(data,file))
sorted(res, key=lambda k: k['score'],reverse=True) 

[{'id': '24emb.csv', 'score': 0.8779039544587771},
 {'id': '68emb.csv', 'score': 0.8705710630591886},
 {'id': '8emb.csv', 'score': 0.8695739063243675},
 {'id': '62emb.csv', 'score': 0.8678714732044934},
 {'id': '35emb.csv', 'score': 0.8642819191161069},
 {'id': '61emb.csv', 'score': 0.8634623557198571},
 {'id': '59emb.csv', 'score': 0.8584177055214488},
 {'id': '52emb.csv', 'score': 0.8583775432442089},
 {'id': '67emb.csv', 'score': 0.8583230494554309},
 {'id': '71emb.csv', 'score': 0.8581514555851563},
 {'id': '50emb.csv', 'score': 0.856687181824969},
 {'id': '20emb.csv', 'score': 0.8533177967894009},
 {'id': '32emb.csv', 'score': 0.8519704780703952},
 {'id': '21emb.csv', 'score': 0.8515719642124727},
 {'id': '60emb.csv', 'score': 0.8510525063508926},
 {'id': '49emb.csv', 'score': 0.849824126083865},
 {'id': '43emb.csv', 'score': 0.8493379355396096},
 {'id': '41emb.csv', 'score': 0.8478204727700385},
 {'id': '66emb.csv', 'score': 0.847253084037583},
 {'id': '7emb.csv', 'score': 0.8464